# Convert SQLite outputs to parquet files with cytotable

## Import libraries

In [1]:
import pathlib
import pandas as pd

# cytotable will merge objects from SQLite file into single cells and save as parquet file
from cytotable import convert, presets

import logging

# Set the logging level to a higher level to avoid outputting unnecessary errors from config file in convert function
logging.getLogger().setLevel(logging.ERROR)

## Set paths and variables

In [2]:
# preset configurations based on typical CellProfiler outputs
preset = "cellprofiler_sqlite_pycytominer"

# update preset to include site metadata and cell counts
joins = presets.config["cellprofiler_sqlite_pycytominer"]["CONFIG_JOINS"].replace(
    "Image_Metadata_Well,",
    "Image_Metadata_Well, Image_Metadata_Site, Image_Count_Cells,",
)

# type of file output from cytotable (currently only parquet)
dest_datatype = "parquet"

# set path to directory with SQLite files
sqlite_dir = pathlib.Path("../2.cellprofiler_processing/cp_output")

# directory for processed data
output_dir = pathlib.Path("data")
output_dir.mkdir(parents=True, exist_ok=True)

plate_names = []

for file_path in sqlite_dir.iterdir():
    plate_names.append(file_path.stem)

# print the plate names and how many plates there are (confirmation)
print(f"There are {len(plate_names)} plates in this dataset. Below are the names:")
for name in plate_names:
    print(name)

There are 4 plates in this dataset. Below are the names:
localhost230405150001
localhost220512140003_KK22-05-198
localhost231120090001
localhost220513100001_KK22-05-198_FactinAdjusted


## Convert SQLite to parquet files

In [3]:
for file_path in sqlite_dir.iterdir():
    output_path = pathlib.Path(f"{output_dir}/converted_profiles/{file_path.stem}_converted.parquet")
    print("Starting conversion with cytotable for plate:", file_path.stem)
    # merge single cells and output as parquet file
    convert(
        source_path=str(file_path),
        dest_path=str(output_path),
        dest_datatype=dest_datatype,
        preset=preset,
        joins=joins,
        chunk_size=5000
    )
    
print("All plates have been converted with cytotable!")

Starting conversion with cytotable for plate: localhost230405150001
Starting conversion with cytotable for plate: localhost220512140003_KK22-05-198
Starting conversion with cytotable for plate: localhost231120090001
Starting conversion with cytotable for plate: localhost220513100001_KK22-05-198_FactinAdjusted
All plates have been converted with cytotable!


# Load in converted profiles to update

In [4]:
# Directory with converted profiles
converted_dir = pathlib.Path(f"{output_dir}/converted_profiles")

for file_path in converted_dir.iterdir():
    # Load the DataFrame from the Parquet file
    df = pd.read_parquet(file_path)
    
    # Drop rows where "Metadata_ImageNumber" is NaN (artifact of cytotable)
    df = df.dropna(subset=["Metadata_ImageNumber"])
    
    # Columns to move to the front
    columns_to_move = ['Cells_Location_Center_X', 'Cells_Location_Center_Y', "Image_Count_Cells"]
    
    # Rearrange columns and add "Metadata" prefix in one line
    df = (df[columns_to_move + [col for col in df.columns if col not in columns_to_move]]
                  .rename(columns=lambda col: 'Metadata_' + col if col in columns_to_move else col))
    
    # Save the processed DataFrame as Parquet in the same path
    df.to_parquet(file_path, index=False)


## Check output to confirm process worked

To confirm the number of single cells is correct, please use any database browser software to see if the number of rows in the "Per_Cells" compartment matches the number of rows in the data frame.

In [5]:
converted_df = pd.read_parquet("./data/converted_profiles/localhost231120090001_converted.parquet")

print(converted_df.shape)
converted_df.head()

(21370, 2016)


,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Image_Metadata_Plate,Image_Metadata_Site,Image_Metadata_Well,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,...,Nuclei_Texture_Variance_Hoechst_3_02_256,Nuclei_Texture_Variance_Hoechst_3_03_256,Nuclei_Texture_Variance_Mitochondria_3_00_256,Nuclei_Texture_Variance_Mitochondria_3_01_256,Nuclei_Texture_Variance_Mitochondria_3_02_256,Nuclei_Texture_Variance_Mitochondria_3_03_256,Nuclei_Texture_Variance_PM_3_00_256,Nuclei_Texture_Variance_PM_3_01_256,Nuclei_Texture_Variance_PM_3_02_256,Nuclei_Texture_Variance_PM_3_03_256
0,520.849209,277.583420,20,1.0,None,f00,B02,6.0,6.0,8.0,...,2.261202,2.234199,0.283552,0.286277,0.281137,0.280018,0.824348,0.831982,0.822502,0.794160
1,313.661110,374.449986,20,1.0,None,f00,B02,7.0,7.0,9.0,...,3.083781,3.047163,0.238799,0.235608,0.262517,0.247103,0.886573,0.890560,0.909833,0.885598
2,709.496383,379.652932,20,1.0,None,f00,B02,8.0,8.0,10.0,...,0.899203,0.919880,0.155762,0.155740,0.164075,0.156480,0.810832,0.835048,0.813655,0.812339
3,869.851378,511.154606,20,1.0,None,f00,B02,9.0,9.0,12.0,...,2.913076,2.911455,0.377669,0.374731,0.363038,0.346197,2.859340,2.973961,2.922192,2.739148
4,588.967372,503.936707,20,1.0,None,f00,B02,10.0,10.0,13.0,...,3.693088,3.715200,0.189286,0.200514,0.192271,0.189659,1.359794,1.338005,1.371597,1.304711
